# Import Statements

In [1]:
import pandas as pd
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import dominate
from dominate.tags import *
import espn_api.football
from espn_api.football import League, Player
import time
import pickle
import uuid
import drawsvg as draw

from python import constants, functions

# Testing and Debugging

In [3]:
league = League(league_id=constants.LEAGUE_ID, year=2023, espn_s2=constants.ESPN_S2, swid=constants.SWID)

In [62]:
players = league.espn_request.get_pro_players()

players = players[281]
p = {
    'player_name':players['fullName'],
    'position':espn_api.football.constant.POSITION_MAP[players['defaultPositionId']],
    'pro_team':espn_api.football.constant.PRO_TEAM_MAP[players['proTeamId']]
}

p

{'player_name': 'Tony Pollard', 'position': 'RB', 'pro_team': 'TEN'}

In [61]:
for i in range(len(players)):
    if players[i]['fullName'] == 'Tony Pollard':
        print(i)


281


In [38]:
X = [30,60,90]
Y = [100,120,135]

width = 300
height = 150

d = draw.Drawing(width=width, height=height)

margin_lr, margin_tb = 20, 30

plot_width = width - (2*margin_lr)
plot_height = height - (2*margin_tb)

X_scaled = [x / plot_width for x in X]
print(X_scaled)

# points = [c for xy in zip(X, Y) for c in xy]
points = [z for z in zip(X,Y)]
d.append(draw.Lines(*points, fill='none', stroke='black'))

d.display_image()

[0.11538461538461539, 0.23076923076923078, 0.34615384615384615]


TypeError: expected an even number of positional arguments x0, y0, x1, y1, ...

In [ ]:
Z = [z for z in zip(X,Y)]


(30, 100) (60, 120) (90, 135)


In [20]:
year = 2023
data = constants.GAME_DATA.loc[constants.GAME_DATA['Year'] == year].copy()
champion = data.loc[(data['Week'] == data['Week'].max()) & (data['Win'] == 1), 'Team'].item()

champ_data = functions.summary_table(data=data, year=year)
champ_data.loc[champ_data['Team'] == champion]

,Year,Team,Record,Ranking,Points For,Points Against,Avg Points For,Avg Margin,Luck Score
0,2023,Haris,12-2,1,1801.34,1456.3,128.67,24.65,10


In [ ]:
champions = []
for year in constants.YEARS:
    data = constants.GAME_DATA.loc[constants.GAME_DATA['Year'] == year].copy()
    champion = data.loc[(data['Week'] == data['Week'].max()) & (data['Win'] == 1), 'Team'].item()
    data = functions.summary_table(data=data, year=year).loc[]

champ: Dante + year: 2018
champ: Dante + year: 2019
champ: Ethan + year: 2020
champ: Andrew + year: 2021
champ: Dante + year: 2022
champ: Haris + year: 2023
champ: Zach + year: 2024


In [ ]:
'''
Function which creates the champion page content

Includes:
---------
* Summary Table for each season
* List of all Champions
'''
container = div(_class='content')
container.add(h1('League Champions'))

champions = constants.GAME_DATA.loc[(constants.GAME_DATA['Week'] == 17) & (constants.GAME_DATA['Win'] == 1), ['Year','Team']].values
data = []
for year, team in champions:
    temp = constants.GAME_DATA.loc[(constants.GAME_DATA['Year'] == year) & (constants.GAME_DATA['Team'] == team) & (constants.GAME_DATA['Playoff Flag'] == False)]
    wins = temp['Win'].sum()
    losses = len(temp) - wins
    record = f'{wins}-{losses}'

    pf = round(temp['Score'].sum(), 2)
    avg_pf = round(pf / len(temp), 2)
    pa = round(temp['Opp Score'].sum(), 2)
    avg_margin = round((pf - pa) / len(temp), 2)

    data.append([year, team, record, pf, avg_pf, avg_margin])

t = pd.DataFrame(data, columns=['Year','Team','Record','Points For','Avg Points For','Avg Margin'])

summary_div = functions.content_container(title='Champions Summary', content=functions.df_to_table(data=t, table_id='champion-summary-table'))

# summary_div = div(_class='content-container')
# summary_title = h2('Champions Summary')
# summary_table = functions.df_to_table(data=t)
# summary_table['id'] = 'champion-summary-table'
# summary_div.add([summary_title, summary_table])

years_div = div(_class='champion-years')
for year, team in champions:
    years_div.add(h3(f'{year} Season'))
    years_div.add(p(team))

container.add(summary_div)
container.add(years_div)

champions
# print(container.render())

array([[2021, 'Andrew'],
       [2022, 'Dante'],
       [2023, 'Haris'],
       [2024, 'Zach']], dtype=object)

In [ ]:
def summary_table(data: pd.DataFrame, year: int, week: int = None, averages: bool = False) -> pd.DataFrame:
    temp = data.loc[(data['Year'] == year) & (data['Playoff Flag'] == False)].copy()

    if week != None:
        temp = temp.loc[(data['Week'] <= week)]

    mean = temp['Score'].mean()
    std = temp['Score'].std() * 0.5

    opp_luck = []
    your_luck = []
    close_luck = []

    for your_score, opp_score in temp[['Score','Opp Score']].values:
        opp_luck.append(functions.opp_luck_score(opp_score=opp_score, mean=mean, std=std))
        your_luck.append(functions.your_luck_score(your_score=your_score, opp_score=opp_score, mean=mean, std=std))
        close_luck.append(functions.close_luck_score(your_score=your_score, opp_score=opp_score))

    temp['Opp Luck Score'] = opp_luck
    temp['Your Luck Score'] = your_luck
    temp['Close Luck Score'] = close_luck

    temp['Luck Score'] = temp[['Opp Luck Score','Your Luck Score','Close Luck Score']].sum(axis=1)

    temp_teams = temp['Team'].unique()
    weekly_standings = []

    for team in temp_teams:
        temp_team = temp.loc[temp['Team'] == team]
        wins = temp_team['Win'].sum()
        losses = temp_team['Win'].eq(0).sum()
        record = f'{wins}-{losses}'

        pf = round(temp_team['Score'].sum(), 2)
        avg_pf = round(pf / len(temp_team), 2)
        pa = round(temp_team['Opp Score'].sum(), 2)
        avg_margin = round((pf - pa) / len(temp_team), 2)
        luck_score = temp_team['Luck Score'].sum()

        weekly_standings.append(
            {
                'Team':team,
                'Wins':wins,
                'Record':record,
                'Points For':pf,
                'Points Against':pa,
                'Avg Points For':avg_pf,
                'Avg Margin':avg_margin,
                'Luck Score':luck_score
            }
        )
        # weekly_standings.append([team, wins, record, pf, pa, avg_pf, avg_margin, luck_score])

    

    # weekly_standings = pd.DataFrame(weekly_standings, columns=['Team','Wins','Record', 'Points For','Points Against','Avg Points For','Avg Margin','Luck Score'])
    weekly_standings = pd.DataFrame(weekly_standings)
    weekly_standings.sort_values(['Wins','Points For'], ascending=False, ignore_index=True, inplace=True)
    weekly_standings['Ranking'] = [i + 1 for i in weekly_standings.index]
    weekly_standings['Year'] = year
    weekly_standings = weekly_standings[['Year','Team','Record','Ranking','Points For','Points Against','Avg Points For','Avg Margin','Luck Score']]

    if averages:
        pf = round(temp['Score'].sum() / len(temp_teams), 2)
        pa = round(temp['Opp Score'].sum() / len(temp_teams), 2)
        luck_score = weekly_standings['Luck Score'].mean()

        avg_row = {
                'Year':'',
                'Team':'',
                'Record':'',
                'Ranking':'',
                'Points For':pf,
                'Points Against':pa,
                'Avg Points For':'',
                'Avg Margin':'',
                'Luck Score':''
        }
        weekly_standings.loc['Averages'] = avg_row

    return weekly_standings

In [17]:
data = constants.GAME_DATA.copy()
data_champs = []

for year in constants.YEARS:
    champ_week = data.loc[data['Year'] == year, 'Week'].max()

    champ = data.loc[(data['Year'] == year) & (data['Week'] == champ_week) & (data['Win'] == 1), 'Team'].item()

    data_champs.append(
        {
            'year':year,
            'champion':champ
        }
    )

data_champs

pd.DataFrame(data_champs)

,year,champion
0,2018,Dante
1,2019,Dante
2,2020,Ethan
3,2021,Andrew
4,2022,Dante
5,2023,Haris
6,2024,Zach


In [2]:
def summary_table(data: pd.DataFrame, year: int, week: int = None) -> pd.DataFrame:
    temp = data.loc[(data['Year'] == year) & (data['Playoff Flag'] == False)].copy()

    if week != None:
        temp = temp.loc[(data['Week'] <= week)]

    mean = temp['Score'].mean()
    std = temp['Score'].std() * 0.5

    opp_luck = []
    your_luck = []
    close_luck = []

    for your_score, opp_score in temp[['Score','Opp Score']].values:
        opp_luck.append(functions.opp_luck_score(opp_score=opp_score, mean=mean, std=std))
        your_luck.append(functions.your_luck_score(your_score=your_score, opp_score=opp_score, mean=mean, std=std))
        close_luck.append(functions.close_luck_score(your_score=your_score, opp_score=opp_score))

    temp['Opp Luck Score'] = opp_luck
    temp['Your Luck Score'] = your_luck
    temp['Close Luck Score'] = close_luck

    temp['Luck Score'] = temp[['Opp Luck Score','Your Luck Score','Close Luck Score']].sum(axis=1)

    champ_week = data.loc[data['Year'] == year, 'Week'].max()
    champ = data.loc[(data['Year'] == year) & (data['Week'] == champ_week) & (data['Win'] == 1), 'Team'].item()

    temp_teams = temp['Team'].unique()
    weekly_standings = []

    for team in temp_teams:
        temp_team = temp.loc[temp['Team'] == team]
        wins = temp_team['Win'].sum()
        losses = temp_team['Win'].eq(0).sum()
        record = f'{wins}-{losses}'

        pf = round(temp_team['Score'].sum(), 2)
        avg_pf = round(pf / len(temp_team), 2)
        pa = round(temp_team['Opp Score'].sum(), 2)
        avg_margin = round((pf - pa) / len(temp_team), 2)
        luck_score = temp_team['Luck Score'].sum()
        
        champ_flag = int(champ == team)

        weekly_standings.append([team, wins, record, pf, pa, avg_pf, avg_margin, luck_score, champ_flag])

    weekly_standings = pd.DataFrame(weekly_standings, columns=['Team','Wins','Record', 'Points For','Points Against','Avg Points For','Avg Margin','Luck Score', 'Champ Flag'])
    weekly_standings.sort_values(['Wins','Points For'], ascending=False, ignore_index=True, inplace=True)
    weekly_standings['Ranking'] = [i + 1 for i in weekly_standings.index]
    weekly_standings['Year'] = year
    weekly_standings = weekly_standings[['Year','Team','Record','Ranking','Points For','Points Against','Avg Points For','Avg Margin','Luck Score', 'Champ Flag']]

    return weekly_standings

In [2]:
t = functions.df_to_table(
    data=functions.summary_table(data=constants.GAME_DATA, year=2024),
    custom_columns=['Year','Team','Record','Points For','Points Against','Avg Points For','Avg Margin','Luck Score'],
    champ_class=True
)

print(t)

<table>
  <thead>
    <tr>
      <th class="year">Year</th>
      <th class="team">Team</th>
      <th class="record">Record</th>
      <th class="points-for">Points For</th>
      <th class="points-against">Points Against</th>
      <th class="avg-points-for">Avg Points For</th>
      <th class="avg-margin">Avg Margin</th>
      <th class="luck-score">Luck Score</th>
    </tr>
  </thead>
  <tbody>
    <tr><td class="year">2024</td><td class="team">Kevin</td><td class="record">11-3</td><td class="points-for">1777.24</td><td class="points-against">1568.64</td><td class="avg-points-for">126.95</td><td class="avg-margin">14.9</td><td class="luck-score">5</td></tr>
    <tr><td class="year">2024</td><td class="team">Andrew</td><td class="record">10-4</td><td class="points-for">1873.18</td><td class="points-against">1681.98</td><td class="avg-points-for">133.8</td><td class="avg-margin">13.66</td><td class="luck-score">-3</td></tr>
    <tr><td class="year">2024</td><td class="team">Carter</t